In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scripts import make_grid as mg

Все функии для модели описаны в этом модуле:

In [3]:
from scripts import poisson_covariance as pc

In [4]:
data = pd.read_pickle('work_catalog_pd')

Модели, которые рассматриваются ниже более требовательны к наличию наблюдений в одной ячейки.
Поэтому возьмем сетку по пространству значительно крупнее.

Разобьем все наблюдения по пространственной сетке на $2\times 2$ и $48$ суток в ячейке по времени. $20$ отсчетов отделим как тренеровочную выборку.

In [5]:
time_split = mg.TXYSplit(data, 48.0, 2, 2)
xyt_matrix = mg.TXYMatrix(time_split, 2, 2)
xyt_matrix_train, xyt_matrix_test = (xyt_matrix[:,:,:20]).copy(), (xyt_matrix[:,:,20:]).copy()

In [6]:
xyt_matrix.shape

(10, 20, 200)

Сформируем матрицу, где для каждой ячейки в каждый момент времени поднимется флаг аномалии.

In [7]:
anomaly_flag = mg.target_anomaly_matrix(xyt_matrix_test, 7.0)
anomaly_x, anomaly_y, anomaly_t = np.where(anomaly_flag != 0)
anomaly_coord = zip(anomaly_x, anomaly_y, anomaly_t)

In [23]:
anomaly_coord

[(1, 6, 8),
 (1, 6, 91),
 (2, 4, 61),
 (2, 5, 18),
 (3, 8, 140),
 (3, 9, 153),
 (3, 10, 160),
 (4, 8, 81),
 (4, 8, 98),
 (4, 8, 119),
 (4, 8, 141),
 (4, 9, 140),
 (4, 9, 143),
 (4, 10, 100),
 (4, 10, 140),
 (5, 8, 18),
 (5, 9, 18),
 (5, 9, 84),
 (5, 9, 121),
 (6, 3, 50),
 (6, 4, 14),
 (6, 7, 7),
 (6, 9, 3),
 (6, 10, 93)]

Рассмотрим пуассоновский процесс количества землетрясений для каждой ячейки, где интенсивность зависит от вектора некоторых параметров $X$ линейно через вектор коэффициентов $\beta$. Получим апостериорное распределение на $\beta$ и прогнозную плотность. Таким образом мы сможем детектировать аномальные значения:
 + $\beta_{t+1}$ относительно распределения $\beta_{t}$
 + невязки между прогноза на количество землетрясений $t+1$ при полученных $x_{t+1}$ по прогнозной плости обученной до момента $t$

Пусть количество $y$ земелтрясений распределено согласно закону:

$$
y\sim Poisson(\mu(x))
$$

Параметризуем интесивность: 

$$
\mu(x) = \exp(\lambda(x))
$$

$$
\lambda(x) = x^T\beta
$$

Введем априорное распределение на $\beta$:

$$
\beta\sim\mathcal{N}(0, \Sigma_p)
$$


Тогда функция правдоподобия вектора количеств земелетрясений $y$ при фиксированных $X$, $\beta$:
    
$$
p(y|X,\beta) = \prod\limits_{i=1}^{N}\dfrac{1}{y_i!}\mu(x_i)^{y_i}\exp(-\mu(x_i)) = \\
\prod\limits_{i=1}^{N}\dfrac{1}{y_i\cdot(y_i-1)}\exp(y_i\lambda(x_i))\exp[-\exp(\lambda(x_i))]
$$

Т.к. мы ввели $\beta\sim\mathcal{N}$, то полное правдоподобие будет неспоряженным. Придется использовать $MCMC$ семплирование для оценки апостериорного распределения, чего хотелось бы избежать. Хочется получить простые формулы пересчета для каждого нового момента $t+1$, т.к. мы будем считать на сетке $200\times400$. К тому же, итоговая цель не точное предсказание числа землетрясений, а предсказание аномально сильных. Поэтому нам необходимо иметь лишь достаточную точность моделей для улавливания предвестников таких событий. Поэтому будем использовать аппроксимацию.

Пусть $\mu\sim\Gamma(a, b)$. $\lambda = \log\mu$ Но логарифм гамма распределения хорошо аппроксимируется нормальным с параметрами:
$$
\log\mu\sim\mathcal{N}(\log{a}+\log{b},a^{-1})
$$

Положим $b=1$ b получаем, что:

$$
\lambda = \log\mu\sim\mathcal{N}(\log{a},a^{-1})
$$

В итоге получаем, лографим правдоподобия проропрционален:

$$
\log p(y|X,\beta)\propto -\dfrac{1}{2}\left([X^T\beta - \log y]\operatorname{diag}y_i)[X^T\beta - \log y]^T\right)
$$

Тогда полное правдоподобие:

$$
\log p(y|X,\beta)p(\beta) \propto -\dfrac{1}{2}\left([X^T\beta - \log y]\operatorname{diag}y_i[X^T\beta - \log y]^T\right) -\dfrac{1}{2}\beta\Sigma_p^{-1}\beta^T
$$

И значит 

$$
p(\beta|X, y)\sim\mathcal{N}(\beta|\mu_{\beta}, \Sigma_{\beta})
$$

Максимизируя квадратичную форму, находим, что

$$
\mu_{\beta} = (X\Sigma^{-1}_yX^T+\Sigma_p^{-1})^{-1}X\Sigma_y^{-1}\log y
$$

$$
\Sigma_{\beta} = (X\Sigma^{-1}_yX^T+\Sigma_p^{-1})^{-1}
$$

$$
X = [x_1\dots x_N],~\Sigma_y = \operatorname{diag}\dfrac{1}{y_i}$$

Выпишем также формулу пересчета $\beta_{t+1}$, при априорном распределении $\beta_t$

$$
p(\beta_{t+1}|\beta_{t})\sim\mathcal{N}(\mu', \Sigma') \\
$$


$$
\mu' = (\Sigma^{-1}_{\beta_t} + x_i\Sigma^{-1}_{y_{t+1}}x_i^T)^{-1}(\Sigma^{-1}_{\beta_t}\mu_{\beta_t}+ x_i\Sigma^{-1}_{y_{t+1}}\log y_{t+1})
$$


$$
\Sigma' = (\Sigma^{-1}_{\beta_t} + x_i\Sigma^{-1}_{y_{t+1}}x_i^T)^{-1}
$$

Теперь необходимо найти предсказательную плотность. 

$$
p(y_{new}|x_{new}, X,y) = \int p(y_{new}|x_{new},\beta)p(\beta|X,y)d\beta
$$

Опуская выкладки, получаем:

$$
p(y_{new}|x_{new}, X,y)\sim\text{NegBin}(r, p) \\
p = \dfrac{e^{\mu_{\lambda}}}{e^{\mu_{\lambda}} + \sigma^{-2}_{\lambda}} \\
r = \sigma^{-2}_{\lambda} \\
\\
\mu_{\lambda} = x_{new}^T\mu_{\beta} \\
\sigma^2_{\lambda} = x_{new}^T\Sigma_{\beta}x_{new}
$$

Заметим, что также была получаенна аппроксимация на распределение $\lambda\sim\mathcal{N}(\mu_{\lambda}, \sigma^2_{\lambda})$

Обучимся сначала на каждой ячейки на $train$ части, т.е. первых $20$ отсчетах сетки.

Далее модели будут различаться только матрицей $X$

### Модель 1

Первая модель будет "изолированная". Будем в качестве регрессоров использовать статистики о магнитуде различных землетрясений только внутри одной ячейки:

$$
x_i = [\min[M(Cell_i)], \operatorname{median}[M(Cell_i)]\max[M(Cell_i)]]
$$

In [8]:
T = 20
t_matrix_train = xyt_matrix_train[6, 10,:]
t_matrix_test = xyt_matrix_test[6, 10,:]
params = pc.BetaStatisticWindowEstimation(t_matrix_train, t_matrix_test, T)

In [9]:
current_sigma, current_mu = params['sigma'][1:], params['mu'][1:]
past_sigma, past_mu = params['sigma'][:-1], params['mu'][:-1]

Рассчитаем вероятность более экстремальных событий, чем $\mathbb{E}_{t+1}\beta_{t+1}$ по распределению $\beta_t$

In [13]:
p_value = pc.ProbCurrentMuInPastVector(past_sigma, past_mu, current_sigma, current_mu)

Рассчитаем $KL$ дивергенцию между распределениями $\beta_{t+1}$ и $\beta_t$

In [15]:
KLFuturePast = pc.KLdivergenceNormalsVector(past_sigma, past_mu, current_sigma, current_mu)
KLPastFuture = pc.KLdivergenceNormalsVector(current_sigma, current_mu, past_sigma, past_mu)

Рассчитаем отношение $\dfrac{\log p_{t+1}(\mathbb{E}_{t+1}\beta)}{\log p_{t}(\mathbb{E}_{t+1}\beta)}$

In [16]:
log_odds = pc.LogOddsRationVector(past_sigma, past_mu, current_sigma, current_mu)

Будем находить распределение $y_{t+1}$ (предсказание) по модели $\beta_t$ при наблюдении $x_{t+1}$. Для этого для каждого $\beta_t$ и $x_{t+1}$ необходимо рассчитать параметры распределение $\lambda_t\sim\mathcal{N}$

In [17]:
x = np.asarray(pc.CellXEstimateVector(t_matrix_test))
lambda_params = pc.LambdaVectorEstimation(params, x)

In [18]:
mode_prediction = pc.ModePredictionYVector(lambda_params['sigma'], lambda_params['mu'])
mean_prediction = pc.ExpectationPredictionYVector(lambda_params['sigma'], lambda_params['mu'])

In [19]:
y_real = pc.CellYEstimateVector(t_matrix_test)
y_real = y_real[1:]

Все работает. Сведем все в одну функцию, которая принимает на вход координты ячейки и выдает матрицу фичей, где номер строки отвечает шагу по времени, за вычетом train и одного наблюдения.

In [24]:
T = 20
t_matrix_train = xyt_matrix_train[6, 10,:]
t_matrix_test = xyt_matrix_test[6, 10,:]
result = cp.MakeDictStatistics(T, t_matrix_train, t_matrix_test)